In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:

# Read Portuguese-Brazil CSV safely
Estoucomfome_data = pd.read_csv("Churn_Clientes.csv",
                 encoding='latin1',   # correct for Portuguese
                 sep=';',             # many PT-BR CSVs use ';' instead of ','
                 engine='python')



In [6]:
data_2 = pd.read_csv("fooddelivery.csv")
print(data_2.head())

   customerId  orderQuantity areaName  OrderDinner  OrderLunch  \
0        5148             34    Hub 3        False        True   
1        5141              9  Benfica         True        True   
2        5135              1  Benfica        False        True   
3        5130              4  Benfica        False        True   
4        5121             16    Hub 3        False        True   

   dessertQuantity  discountQuantity  drinkQuantity  plateQuantity  \
0                0                 0              1             65   
1                0                 0              0             21   
2                0                 0              1              1   
3                1                 0              0              8   
4                0                19              0             22   

   noplateQuantity  ...  email_ok  language  sms_ok   plan  promocode gender  \
0                1  ...      True        pt    True  False      False      M   
1                5  ..

In [10]:
# Rename columns to English (CamelCase)
Estoucomfome_data.rename(columns={
    'ClienteId': 'Client_ID',
    'DataUltimaTransacao': 'Last_Transaction_Date',
    'DataExtracao': 'Extraction_Date',
    'Score_Credito': 'Credit_Score',
    'Estado': 'State',
    'Gênero': 'Gender',
    'Idade': 'Age',
    'Tempo_Cliente': 'Customer_Tenure_Months',
    'Limite_Credito_Mercado': 'Market_Credit_Limit',
    'Qte_Categorias': 'Product_Categories_Count',
    'Usa_Cartao_Credito': 'Uses_Credit_Card',
    'Programa_Fidelidade': 'Loyalty_Program',
    'Sum_Pedidos_Acumulados': 'Total_Spent_Last3M',
    'Churn': 'Churn'
}, inplace=True)

# Verify changes
print(Estoucomfome_data.columns)
print(Estoucomfome_data.head())

Index(['ClientId', 'Extraction_Date', 'Credit_Score', 'State', 'GÃªnero',
       'Age', 'Customer_Tenure_Months', 'limite_credito_mercado',
       'Product_Categories_Count', 'Uses_Credit_Card', 'Loyalty_Program',
       'sum_pedidos_acumulado', 'Last_Transaction_Date', 'churn'],
      dtype='object')
   ClientId Extraction_Date  Credit_Score           State   GÃªnero  Age  \
0    345568      30/06/2019           619       São Paulo  Feminino   42   
1    345569      30/06/2019           608  Rio de Janeiro  Feminino   41   
2    345570      30/06/2019           502       São Paulo  Feminino   42   
3    345571      30/06/2019           699       São Paulo  Feminino   39   
4    345572      30/06/2019           850  Rio de Janeiro  Feminino   43   

   Customer_Tenure_Months limite_credito_mercado  Product_Categories_Count  \
0                       2                     0.                         1   
1                       1                 838.08                         1   
2     